# Dependency Inversion Principle (DIP)

> “High‑level modules should not depend on low‑level modules. Both should depend on abstractions.” – Robert C. Martin

## 1. Concept
* **DIP** is not just Dependency Injection (DI). It inverts the direction of dependency toward **abstractions**.
* High‑level policies (business rules) do not know details of low‑level mechanics (DB, file system, HTTP).

### IoC vs. DI vs. Containers
| Term | Essence | Relation to DIP |
|------|---------|-----------------|
| Inversion of Control (IoC) | Framework owns the program flow. | DIP realises IoC for dependencies. |
| Dependency Injection (DI) | Passing collaborators instead of creating them. | Main tactic implementing DIP. |
| Dependency Container | Tool that automates DI wiring. | Convenience ‑ optional for DIP. |

## 2. Problematic design without DIP

In [ ]:
function generateId(): string {
  return `${Date.now().toString(36)}-${Math.random().toString(36).slice(2, 8)}`;
}

// BAD: High‑level service hard‑codes concrete repo
class SqlUserRepository {
  save(user: {id: string; name: string}) {
    console.log("INSERT INTO users ...", user);
  }
}

class UserRegistrationService {
  private repo = new SqlUserRepository();  // concrete dependency

  register(name: string) {
    const user = {id: generateId(), name};
    this.repo.save(user);
  }
}

new UserRegistrationService().register("Alice");

INSERT INTO users ... { id: 'm9ozg1n8-lafrk8', name: 'Alice' }


### Issues
* Tightly coupled to SQL API – cannot switch storage without editing service.
* Hard to unit‑test: needs DB access.
* Violates Open/Closed Principle.

## 3. Refactor applying DIP

In [ ]:
function generateId(): string {
  return `${Date.now().toString(36)}-${Math.random().toString(36).slice(2, 8)}`;
}

// 1️⃣ Abstraction
interface UserRepository {
  save(user: {id: string; name: string}): void;
}

// 2️⃣ Concrete implementations
class SqlUserRepository implements UserRepository {
  save(user: {id: string; name: string}) {
    console.log("SQL save", user);
  }
}

class InMemoryUserRepository implements UserRepository {
  private data: any[] = [];
  save(user: {id: string; name: string}) {
    this.data.push(user);
  }
}

// 3️⃣ High‑level module depends on abstraction
class UserRegistrationService {
  constructor(private repo: UserRepository) {}

  register(name: string) {
    const user = {id: generateId(), name};
    this.repo.save(user);
  }
}

// 4️⃣ Composition root wiring
const prodService = new UserRegistrationService(new SqlUserRepository());
const testService = new UserRegistrationService(new InMemoryUserRepository());

prodService.register("Bob");
testService.register("Test");

SQL save { id: 'm9ozhtri-ua6fg0', name: 'Bob' }


### Benefits
* Swap implementations with no code changes in high‑level module.
* Inject fakes/mocks for fast tests.
* Clear boundaries between layers.

## 4. Trade‑offs
* More abstractions & wiring.
* Need a composition root or container.
* Poorly designed abstractions can leak details.

## 5. Spotting DIP violations – quick checklist
1. High‑level code constructs low‑level classes with `new`.
2. Business logic imports DB/HTTP libraries directly.
3. Tests require infrastructure services running.
4. Layer boundaries are crossed by concrete types.

## 6. Related GoF patterns
* **Abstract Factory / Factory Method** – create objects without binding to concretes.
* **Observer** – publishers depend on subscriber interfaces.
* **Strategy** – algorithms depend on abstraction, client swaps them.

## 7. References
* Robert C. Martin – *Agile Software Development* (2002)
* https://principles.dev/#dip